<h1>Chapter 8 - Semantic Search and Retrieval-Augmented Generation</h1>
<i>Exploring a vital part of LLMs, search.</i>

<a href="https://www.amazon.com/Hands-Large-Language-Models-Understanding/dp/1098150961"><img src="https://img.shields.io/badge/Buy%20the%20Book!-grey?logo=amazon"></a>
<a href="https://www.oreilly.com/library/view/hands-on-large-language/9781098150952/"><img src="https://img.shields.io/badge/O'Reilly-white.svg?logo=data:image/svg%2bxml;base64,PHN2ZyB3aWR0aD0iMzQiIGhlaWdodD0iMjciIHZpZXdCb3g9IjAgMCAzNCAyNyIgZmlsbD0ibm9uZSIgeG1sbnM9Imh0dHA6Ly93d3cudzMub3JnLzIwMDAvc3ZnIj4KPGNpcmNsZSBjeD0iMTMiIGN5PSIxNCIgcj0iMTEiIHN0cm9rZT0iI0Q0MDEwMSIgc3Ryb2tlLXdpZHRoPSI0Ii8+CjxjaXJjbGUgY3g9IjMwLjUiIGN5PSIzLjUiIHI9IjMuNSIgZmlsbD0iI0Q0MDEwMSIvPgo8L3N2Zz4K"></a>
<a href="https://github.com/HandsOnLLM/Hands-On-Large-Language-Models"><img src="https://img.shields.io/badge/GitHub%20Repository-black?logo=github"></a>
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/HandsOnLLM/Hands-On-Large-Language-Models/blob/main/chapter08/Chapter%208%20-%20Semantic%20Search.ipynb)

---

This notebook is for Chapter 8 of the [Hands-On Large Language Models](https://www.amazon.com/Hands-Large-Language-Models-Understanding/dp/1098150961) book by [Jay Alammar](https://www.linkedin.com/in/jalammar) and [Maarten Grootendorst](https://www.linkedin.com/in/mgrootendorst/).

---

<a href="https://www.amazon.com/Hands-Large-Language-Models-Understanding/dp/1098150961">
<img src="https://raw.githubusercontent.com/HandsOnLLM/Hands-On-Large-Language-Models/main/images/book_cover.png" width="350"/></a>


### [OPTIONAL] - Installing Packages on <img src="https://colab.google/static/images/icons/colab.png" width=100>

If you are viewing this notebook on Google Colab (or any other cloud vendor), you need to **uncomment and run** the following codeblock to install the dependencies for this chapter:

---

💡 **NOTE**: We will want to use a GPU to run the examples in this notebook. In Google Colab, go to
**Runtime > Change runtime type > Hardware accelerator > GPU > GPU type > T4**.

---


In [1]:
# %%capture
# !pip install langchain==0.2.5 faiss-cpu==1.8.0 cohere==5.5.8 langchain-community==0.2.5 rank_bm25==0.2.2 sentence-transformers==3.0.1
# !pip install llama-cpp-python==0.2.78  --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu124

## IMPORTANT: Make sure to restart the session after installing the packages above.

# Dense Retrieval Example


## 1. Getting the text archive and chunking it


In [2]:
text = """
Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan.
It stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine.
Set in a dystopian future where humanity is struggling to survive, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for mankind.

Brothers Christopher and Jonathan Nolan wrote the screenplay, which had its origins in a script Jonathan developed in 2007.
Caltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar.
Cinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm.
Principal photography began in late 2013 and took place in Alberta, Iceland, and Los Angeles.
Interstellar uses extensive practical and miniature effects and the company Double Negative created additional digital effects.

Interstellar premiered on October 26, 2014, in Los Angeles.
In the United States, it was first released on film stock, expanding to venues using digital projectors.
The film had a worldwide gross over $677 million (and $773 million with subsequent re-releases), making it the tenth-highest grossing film of 2014.
It received acclaim for its performances, direction, screenplay, musical score, visual effects, ambition, themes, and emotional weight.
It has also received praise from many astronomers for its scientific accuracy and portrayal of theoretical astrophysics. Since its premiere, Interstellar gained a cult following,[5] and now is regarded by many sci-fi experts as one of the best science-fiction films of all time.
Interstellar was nominated for five awards at the 87th Academy Awards, winning Best Visual Effects, and received numerous other accolades"""

# Split into a list of sentences
texts = text.split('.')

# Clean up to remove empty spaces and new lines
texts = [t.strip(' \n') for t in texts]

## 2. Embedding the Text Chunks, Creating Vector DB, Add and Query Data



In [3]:
from langchain_huggingface import HuggingFaceEmbeddings
from tqdm import tqdm

embedding_model = HuggingFaceEmbeddings(
    model_name='BAAI/bge-small-en-v1.5'
)

c:\Users\USER\Desktop\Neural Networks Repo\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Using Chroma's native API (will use LangChain Version after)

In [4]:
import chromadb
from chromadb import Documents, EmbeddingFunction, Embeddings

class MyEmbeddingFunction(EmbeddingFunction):
    def __call__(self, input: Documents) -> Embeddings:
        embeddings = embedding_model.embed_documents(input)
        return embeddings
    
embedding_function = MyEmbeddingFunction()

chroma_client = chromadb.PersistentClient(path="./chroma_db")
try:
    chroma_client.delete_collection(name="my_collection")
except:
    pass
collection = chroma_client.create_collection(
    name="my_collection",
    embedding_function=embedding_function
)
collection.add(
    ids=[str(id) for id in range(len(texts))], 
    documents=texts
    )

C:\Users\USER\AppData\Local\Temp\ipykernel_104728\624235270.py:9: DeprecationWarning: The class MyEmbeddingFunction does not implement __init__. This will be required in a future version.
  embedding_function = MyEmbeddingFunction()


In [5]:
len(embedding_function(texts))  # should be 15

15

In [6]:
# cannot handle too verbose of a query yet. need intermediate LLM to change query to more concise version later. Use concise query for now.
collection.query(query_texts='how precise was the science', n_results=3)

{'ids': [['12', '4', '7']],
 'embeddings': None,
 'documents': [['It has also received praise from many astronomers for its scientific accuracy and portrayal of theoretical astrophysics',
   'Caltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar',
   'Interstellar uses extensive practical and miniature effects and the company Double Negative created additional digital effects']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[None, None, None]],
 'distances': [[0.6178343892097473, 0.8098766803741455, 0.9237760901451111]]}

In [7]:
import pandas as pd
def search(query, number_of_results=3):
  # 1. Get the query's embedding
  query_embed = embedding_function([query])[0]

  # 2. Retrieve the nearest neighbors
  search_result = collection.query(query_texts=[query], n_results=number_of_results)

  # 3. Format the results
  results = pd.DataFrame(data={'texts': search_result['documents'][0],
                              'distance': search_result['distances'][0]})

  # 4. Print and return the results
  print(f"Query:'{query}'\nNearest neighbors:")
  return results

In [8]:
search("how precise was the science", number_of_results=3) # same as the original notebook.

Query:'how precise was the science'
Nearest neighbors:


,texts,distance
0,It has also received praise from many astronom...,0.617834
1,Caltech theoretical physicist and 2017 Nobel l...,0.809877
2,Interstellar uses extensive practical and mini...,0.923776


### Using Langchain-chroma

In [9]:
from langchain_chroma import Chroma
try:
    chroma_client.delete_collection(name="example_collection")
except:
    pass

vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embedding_model,
    persist_directory="./chroma_langchain_db",
)

In [10]:
from langchain_core.documents import Document
# document_1 = Document(
#     page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
#     metadata={"source": "tweet"},
#     id=1,
# )
documents = [Document(page_content=text) for text in texts]
uuids = [str(id) for id in range(len(documents))]
vector_store.add_documents(documents=documents, ids=uuids)

['0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14']

In [11]:
results = vector_store.similarity_search("how precise was the science", k=3)
results

[Document(id='12', metadata={}, page_content='It has also received praise from many astronomers for its scientific accuracy and portrayal of theoretical astrophysics'),
 Document(id='4', metadata={}, page_content='Caltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar'),
 Document(id='7', metadata={}, page_content='Interstellar uses extensive practical and miniature effects and the company Double Negative created additional digital effects')]

In [12]:
results = vector_store.similarity_search_with_score("how precise was the science", k=3)
results

[(Document(id='12', metadata={}, page_content='It has also received praise from many astronomers for its scientific accuracy and portrayal of theoretical astrophysics'),
  0.6178343892097473),
 (Document(id='4', metadata={}, page_content='Caltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar'),
  0.8098766803741455),
 (Document(id='7', metadata={}, page_content='Interstellar uses extensive practical and miniature effects and the company Double Negative created additional digital effects'),
  0.9237760901451111)]

In [13]:
results = vector_store.similarity_search_by_vector_with_relevance_scores(embedding=embedding_model.embed_query("how precise was the science"), k=3)
results

[(Document(id='12', metadata={}, page_content='It has also received praise from many astronomers for its scientific accuracy and portrayal of theoretical astrophysics'),
  0.6178343892097473),
 (Document(id='4', metadata={}, page_content='Caltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar'),
  0.8098766803741455),
 (Document(id='7', metadata={}, page_content='Interstellar uses extensive practical and miniature effects and the company Double Negative created additional digital effects'),
  0.9237760901451111)]

In [14]:
import pandas as pd
def search_langchain(query, number_of_results=3):
  # 1. Get the query's embedding
  query_embed = embedding_model.embed_query(query)

  # 2. Retrieve the nearest neighbors
  search_result = vector_store.similarity_search_by_vector_with_relevance_scores(embedding=query_embed, k=number_of_results)

  # 3. Format the results
  results = pd.DataFrame(data={'texts': [doc[0].page_content for doc in search_result],
                              'distance': [doc[1] for doc in search_result]})

  # 4. Print and return the results
  print(f"Query:'{query}'\nNearest neighbors:")
  return results

In [15]:
search_langchain("how precise was the science", number_of_results=3)

Query:'how precise was the science'
Nearest neighbors:


,texts,distance
0,It has also received praise from many astronom...,0.617834
1,Caltech theoretical physicist and 2017 Nobel l...,0.809877
2,Interstellar uses extensive practical and mini...,0.923776


In [16]:
from rank_bm25 import BM25Okapi
from sklearn.feature_extraction import _stop_words
import string

def bm25_tokenizer(text):
    tokenized_doc = []
    for token in text.lower().split():
        token = token.strip(string.punctuation)

        if len(token) > 0 and token not in _stop_words.ENGLISH_STOP_WORDS:
            tokenized_doc.append(token)
    return tokenized_doc

In [17]:
from tqdm import tqdm

tokenized_corpus = []
for passage in tqdm(texts):
    tokenized_corpus.append(bm25_tokenizer(passage))

bm25 = BM25Okapi(tokenized_corpus)

100%|██████████| 15/15 [00:00<00:00, 75527.68it/s]


In [18]:
import numpy as np
def keyword_search(query, top_k=3, num_candidates=15):
    print("Input question:", query)

    ##### BM25 search (lexical search) #####
    bm25_scores = bm25.get_scores(bm25_tokenizer(query))
    top_n = np.argpartition(bm25_scores, -num_candidates)[-num_candidates:]
    bm25_hits = [{'corpus_id': idx, 'score': bm25_scores[idx]} for idx in top_n]
    bm25_hits = sorted(bm25_hits, key=lambda x: x['score'], reverse=True)

    print(f"Top-3 lexical search (BM25) hits")
    for hit in bm25_hits[0:top_k]:
        print("\t{:.3f}\t{}".format(hit['score'], texts[hit['corpus_id']].replace("\n", " ")))


In [19]:
keyword_search(query = "how precise was the science")

Input question: how precise was the science
Top-3 lexical search (BM25) hits
	1.789	Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan
	1.373	Caltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar
	0.000	It stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine


## Caveats of Dense Retrieval


In [20]:
query = "What is the mass of the moon?"
results = search(query)
results

Query:'What is the mass of the moon?'
Nearest neighbors:


,texts,distance
0,It has also received praise from many astronom...,0.978440
1,Caltech theoretical physicist and 2017 Nobel l...,1.031753
2,"Interstellar premiered on October 26, 2014, in...",1.120618


# Reranking Example


Need to rewrite with open source models below.

In [21]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

model = AutoModelForSequenceClassification.from_pretrained('cross-encoder/ms-marco-MiniLM-L6-v2')
tokenizer = AutoTokenizer.from_pretrained('cross-encoder/ms-marco-MiniLM-L6-v2')

features = tokenizer(['How many people live in Berlin?', 'How many people live in Berlin?'], ['Berlin has a population of 3,520,031 registered inhabitants in an area of 891.82 square kilometers.', 'New York City is famous for the Metropolitan Museum of Art.'],  padding=True, truncation=True, return_tensors="pt")

model.eval()
with torch.no_grad():
    scores = model(**features).logits
    print(scores.flatten())
    print(torch.argsort(scores, dim=0, descending=True))

tensor([  8.8459, -11.2456])
tensor([[0],
        [1]])


In [22]:
import pandas as pd
def search_with_reranking(query, number_of_results=3):
    # 1. Retrieve the nearest neighbors
    search_result = collection.query(query_texts=[query], n_results=number_of_results)

    documents = search_result['documents'][0]
    # Rerank the documents
    features = tokenizer([query for _ in documents], documents,  padding=True, truncation=True, return_tensors="pt")
    model.eval()
    with torch.no_grad():
        scores = model(**features).logits
    documents = [documents[i] for i in torch.argsort(scores, dim=0, descending=True).squeeze().tolist()]
    distances = [search_result['distances'][0][i] for i in torch.argsort(scores, dim=0, descending=True).squeeze().tolist()]

    # 2. Format the results
    results = pd.DataFrame(data={'texts': documents,
                                'distance': distances})

    # 3. Print and return the results
    print(f"Query:'{query}'\nNearest neighbors:")
    return results

In [23]:
query = "how precise was the science"
search_with_reranking(query=query, number_of_results=3)

Query:'how precise was the science'
Nearest neighbors:


,texts,distance
0,It has also received praise from many astronom...,0.617834
1,Caltech theoretical physicist and 2017 Nobel l...,0.809877
2,Interstellar uses extensive practical and mini...,0.923776


In [24]:
def keyword_and_reranking_search(query, top_k=3, num_candidates=10):
    print("Input question:", query)

    ##### BM25 search (lexical search) #####
    bm25_scores = bm25.get_scores(bm25_tokenizer(query))
    top_n = np.argpartition(bm25_scores, -num_candidates)[-num_candidates:]
    bm25_hits = [{'corpus_id': idx, 'score': bm25_scores[idx]} for idx in top_n]
    bm25_hits = sorted(bm25_hits, key=lambda x: x['score'], reverse=True)

    print(f"Top-3 lexical search (BM25) hits")
    for hit in bm25_hits[0:top_k]:
        print("\t{:.3f}\t{}".format(hit['score'], texts[hit['corpus_id']].replace("\n", " ")))

    #Add re-ranking
    docs = [texts[hit['corpus_id']] for hit in bm25_hits]

    features = tokenizer([query for _ in docs], docs,  padding=True, truncation=True, return_tensors="pt")
    model.eval()
    with torch.no_grad():
        scores = model(**features).logits
    bm25_hits_reranked = [bm25_hits[i] for i in torch.argsort(scores, dim=0, descending=True).squeeze().tolist()]

    print(f"\nTop-3 hits by rank-API ({len(bm25_hits)} BM25 hits re-ranked)")
    for hit in bm25_hits_reranked[0:top_k]:
        print("\t{:.3f}\t{}".format(hit['score'], texts[hit['corpus_id']].replace("\n", " ")))


In [25]:
keyword_and_reranking_search(query = "location of filming")

Input question: location of filming
Top-3 lexical search (BM25) hits
	0.000	Cinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm
	0.000	Principal photography began in late 2013 and took place in Alberta, Iceland, and Los Angeles
	0.000	Interstellar uses extensive practical and miniature effects and the company Double Negative created additional digital effects

Top-3 hits by rank-API (10 BM25 hits re-ranked)
	0.000	Principal photography began in late 2013 and took place in Alberta, Iceland, and Los Angeles
	0.000	Interstellar premiered on October 26, 2014, in Los Angeles
	0.000	Cinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm


# Retrieval-Augmented Generation

## Example: Grounded Generation with an LLM API


Not covered here, see OG notebook.

## Example: RAG with Local Models


### Loading the Generation Model


In [26]:
#!wget https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-q4.gguf

In [27]:
#from langchain import LlamaCpp # defecated
from langchain_community.llms import LlamaCpp
# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path="Phi-3-mini-4k-instruct-q4.gguf",
    n_gpu_layers=-1,
    max_tokens=500,
    n_ctx=2048,
    seed=42,
    verbose=False
)

llama_context: n_batch is less than GGML_KQ_MASK_PAD - increasing to 64
llama_context: n_ctx_per_seq (2048) < n_ctx_train (4096) -- the full capacity of the model will not be utilized


### The RAG Prompt


THESE ARE DEFECATED:
* from langchain import PromptTemplate
* from langchain.chains import RetrievalQA

#### see https://python.langchain.com/docs/versions/migrating_chains/retrieval_qa/

In [28]:
from langchain import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
# Create a prompt template
# must have {context} since below we pass context to it.
template = """<|user|>
Relevant information:
{context}

Provide a concise answer the following question using the relevant information provided above:
{question}<|end|>
<|assistant|>"""
prompt = PromptTemplate(
    template=template,
    input_variables=["context", "question"]
)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


qa_chain = (
    {
        "context": vector_store.as_retriever() | format_docs,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

Note, no memory as we did not incorporate it. To have memory and be able to retrieve stuff, use retrieval as a tool instead of a pipeline.<br>
See: https://langchain-ai.github.io/langgraph/tutorials/get-started/2-add-tools/#6-define-the-conditional_edges

In [29]:
qa_chain.invoke('My name is Kelvin.')

" While Kelvin shares no direct connection to the information provided, Interstellar, a highly acclaimed sci-fi film directed by Christopher Nolan and featuring theoretical physicist Kip Thorne's consultations on scientific accuracy, might have captured your interest in epic science fiction.\n\nHowever, as this is not directly related to Kelvin (assuming you are asking for a connection based on the given data), no specific information can be provided about him within these details. If there's more context regarding who Kelvin is or his interests, I could provide further insights relevant to your inquiry."

In [30]:
qa_chain.invoke('How much monetary value was generated from this movie?')

' The movie generated over $677 million worldwide, with a total of $773 million after subsequent re-releases.'

In [31]:
qa_chain.invoke('What is my name again?')

' Your name is Christopher Nolan.'

#### More verbose version

In [38]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
# must have {context} and {input}.
template = """<|user|>
Relevant information:
{context}

Provide a concise answer the following question using the relevant information provided above:
{input}<|end|>
<|assistant|>"""
prompt = PromptTemplate(
    template=template,
    input_variables=["context", "input"]
)
combine_docs_chain = create_stuff_documents_chain(llm, prompt=prompt)
rag_chain = create_retrieval_chain(vector_store.as_retriever(), combine_docs_chain)

rag_chain.invoke({"input": "How much monetary value was generated from this movie?"})

{'input': 'How much monetary value was generated from this movie?',
 'context': [Document(id='10', metadata={}, page_content='The film had a worldwide gross over $677 million (and $773 million with subsequent re-releases), making it the tenth-highest grossing film of 2014'),
  Document(id='11', metadata={}, page_content='It received acclaim for its performances, direction, screenplay, musical score, visual effects, ambition, themes, and emotional weight'),
  Document(id='5', metadata={}, page_content='Cinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm'),
  Document(id='9', metadata={}, page_content='In the United States, it was first released on film stock, expanding to venues using digital projectors')],
 'answer': ' The movie generated a worldwide gross of over $677 million, with an additional $773 million from subsequent re-releases.'}

### **NOTE:** Recall the current RAG setup is 
1) retrieval first (e.g. dense retrieval)
2) LLM read contents and generate.

In the book, sometimes when the search query is too verbose, we may not retrieve relevant enough documents in step 1).<br>For example:

In [ ]:
query = """
We have an essay due tomorrow. We have to write about some facts about the movie Interstellar.
I love talking about money. I could write about them. But I could also write about the science.
Maybe. Let's do money. How much money did the movie generate?
"""
qa_chain.invoke(query)

' The provided information does not include specific details about the movie "Interstellar"\'s financial earnings or revenue generation. It primarily focuses on the film\'s production, themes, and critical acclaim in various aspects such as performances, direction, screenplay, music score, visual effects, ambition, and emotional weight. Therefore, to answer your question about the movie\'s earnings, additional research would be required beyond the given details.'

To solve this, it is a good idea to use an LLM to rewrite the query into one that aids the retrieval step.

In [44]:
llm_summarizer = LlamaCpp(
    model_path="Phi-3-mini-4k-instruct-q4.gguf",
    n_gpu_layers=-1,
    max_tokens=500,
    n_ctx=2048,
    seed=42,
    verbose=False
)
template = """<|user|>
Your task is to extract the relevant information from the question below to use as a search query:
{question}<|end|>
<|assistant|>"""
question_prompt = PromptTemplate(
    template=template,
    input_variables=["question"]
)

def print_query(query):
    print("Original question:", query)
    concise_query = question_prompt.format(question=query)
    print("Concise query for search:", llm_summarizer.predict(concise_query))
    return query

summarization_chain = (
    question_prompt
    | llm_summarizer | print_query
    | {"context": vector_store.as_retriever() | format_docs, "input": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

llama_context: n_batch is less than GGML_KQ_MASK_PAD - increasing to 64
llama_context: n_ctx_per_seq (2048) < n_ctx_train (4096) -- the full capacity of the model will not be utilized


In [ ]:
query = """
We have an essay due tomorrow. We have to write about some facts about the movie Interstellar.
I love talking about money. I could write about them. But I could also write about the science.
Maybe. Let's do money. How much money did the movie generate?
"""
summarization_chain.invoke(query)

Original question:  search query: "Interstellar movie box office earnings"
Concise query for search:  search query: "Interstellar 2014 box office revenue"


" I'm sorry, but the provided information does not include specific details about Interstellar's box office earnings. However, given its critical acclaim and cult following, it can be inferred that the film was financially successful to a certain extent. For precise figures on its box office performance, further research would be necessary."

In [58]:
qa_chain.invoke('"Interstellar" box office revenue, earnings')

' The provided information does not include specific details on "Interstellar" box office revenue or earnings.'

In [71]:
rag_chain.invoke({"input":'Interstellar money generated'})

{'input': 'Interstellar money generated',
 'context': [Document(id='7', metadata={}, page_content='Interstellar uses extensive practical and miniature effects and the company Double Negative created additional digital effects'),
  Document(id='8', metadata={}, page_content='Interstellar premiered on October 26, 2014, in Los Angeles'),
  Document(id='0', metadata={}, page_content='Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan'),
  Document(id='13', metadata={}, page_content='Since its premiere, Interstellar gained a cult following,[5] and now is regarded by many sci-fi experts as one of the best science-fiction films of all time')],
 'answer': ' The provided information does not include specific financial data on the money generated by "Interstellar." However, it is a critically acclaimed film that premiered in 2014 and has garnered a cult following over time. For detailed box office earnings or revenue figures, additional resea

In [ ]:
rag_chain.invoke({"input":'money generated'})

{'input': 'money generated',
 'context': [Document(id='9', metadata={}, page_content='In the United States, it was first released on film stock, expanding to venues using digital projectors'),
  Document(id='7', metadata={}, page_content='Interstellar uses extensive practical and miniature effects and the company Double Negative created additional digital effects'),
  Document(id='10', metadata={}, page_content='The film had a worldwide gross over $677 million (and $773 million with subsequent re-releases), making it the tenth-highest grossing film of 2014'),
  Document(id='4', metadata={}, page_content='Caltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar')],
 'answer': ' Interstellar generated a worldwide gross of over $677 million, with an additional $773 million from subsequent re-releases.'}

In [ ]:
query = """
We have an essay due tomorrow. We have to write about some facts about the movie.
I love talking about money. I could write about them. But I could also write about the science.
Maybe. Let's do money. How much money did the movie generate?
"""
summarization_chain.invoke(query)

Original question:  search query: "movie box office gross" or "film revenue"
Concise query for search:  search query: "box office revenue of movies" or "film gross income"


' The movie "Interstellar" grossed over $677 million worldwide, making it the tenth-highest earning film of 2014. It was also nominated for five awards at the 87th Academy Awards and won Best Visual Effects.'